# Tutoriel Python - MDSF 2018

Ce tutoriel a pour but de guider les personnes souhaitant utiliser Python pour participer au challenge.

Il comporte 5 étapes :

1. Import des données
2. Analyse descriptive
3. Préparation des données
4. Création d’un modèle
5. Calcul des prédictions et soumissions

# Import des données

Avant de rentrer dans le vif du sujet, installons les packages nécessaires pour ce tutoriel :

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
pd.set_option('display.max_columns', 500)

In [4]:
%%time
X_train = pd.read_csv("../data_challenge/X_train.csv", index_col=0, error_bad_lines=False)
X_test = pd.read_csv("../data_challenge/X_test.csv", index_col=0, error_bad_lines=False)
y_train = pd.read_csv("../data_challenge/y_train.csv", index_col=0)

CPU times: user 154 ms, sys: 26.9 ms, total: 181 ms
Wall time: 220 ms


b'Skipping line 2168: expected 31 fields, saw 33\nSkipping line 4822: expected 31 fields, saw 37\nSkipping line 4859: expected 31 fields, saw 37\nSkipping line 7342: expected 31 fields, saw 37\n'


In [5]:
print("Dimension X_train:", X_train.shape)
print("Dimension X_test:", X_test.shape)

Dimension X_train: (8880, 30)
Dimension X_test: (2960, 30)


In [6]:
X_train.head(3)

,nb_images,longueur_image,largeur_image,url_image,description_produit,taille,matiere,age,garantie,annee,couleur,largeur_produit,wifi,etat,longueur_produit,pointure,vintage,marque,auteur,editions,hauteur_produit,poids,prix,categorie,sous_categorie_1,sous_categorie_2,sous_categorie_3,sous_categorie_4,nom_produit,nom_magasin
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,3,3458.0,2552.0,https://d1kvfoyrif6wzg.cloudfront.net/assets/i...,Superbe petit top bustier avec explosion de co...,44,100 % polyester,NaN,NaN,NaN,Multicolore,NaN,NaN,bon état,NaN,NaN,False,NaN,NaN,NaN,NaN,200.0,4.5,mode,"tops, t-shirts, débardeurs femme",NaN,NaN,NaN,Top bustier multicolore,Emmaüs 88 Neufchateau
1,2,2486.0,2254.0,https://d1kvfoyrif6wzg.cloudfront.net/assets/i...,"Radio ITT Océnic Flirt, année 70\nPour déco",NaN,Plastique,NaN,NaN,NaN,Jaune,NaN,NaN,en l'état,NaN,NaN,True,ITT Océanic,NaN,NaN,NaN,1000.0,15.0,mobilier - deco,bibelots et objets déco,NaN,NaN,NaN,Radio ITT Océanic,Communauté Emmaüs Thouars (magasin Parthenay)
2,3,1536.0,1536.0,https://d1kvfoyrif6wzg.cloudfront.net/assets/i...,Veste boléro à manches courtes NÛMPH. Gris chi...,40,"Polyester, coton, laine",NaN,NaN,NaN,Gris,NaN,NaN,neuf,NaN,NaN,False,Nûmph,NaN,NaN,NaN,360.0,16.0,label selection,mode,mode femme,NaN,NaN,NaN,Label Emmaüs Chambéry


In [21]:
y_train.head(3)

,delai_vente
id,
0,2
1,1
2,1


# Analyse descriptive

## Structure des datasets

Le dataset train comporte les caractéristiques et délai de vente de **8880 objets** vendus sur le site Emmaus. C’est ce dataset que nous allons utiliser pour créer un modèle. Chaque objet est décrit par une observation de X variables. Ces variables sont décrites dans le fichier ```description.pdf``` présent dans la clef USB.

Le dataset test comporte les caractéristiques des **2960 objets** dont il faut prédire le délai de vente. A la différence du train, le délai de vente n’est bien sûr pas renseigné et une colonne ```id``` a ete rajoutée pour identifier les prédictions pendant l’étape de soumission.

In [7]:
X_train.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
nb_images,8880,NaN,NaN,NaN,3.63345,2.04857,0,2,3,5,29
longueur_image,8823,NaN,NaN,NaN,1807.82,1025.25,58,1000,1536,2448,5472
largeur_image,8823,NaN,NaN,NaN,1801.77,1101.21,64,970.5,1536,2448,5472
url_image,8823,8775,https://d1kvfoyrif6wzg.cloudfront.net/assets/i...,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
description_produit,8880,8836,"Relié, 48 pages, couverture usagée",6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
taille,2414,33,38,402,NaN,NaN,NaN,NaN,NaN,NaN,NaN
matiere,3947,1722,Coton,144,NaN,NaN,NaN,NaN,NaN,NaN,NaN
age,120,18,4a,14,NaN,NaN,NaN,NaN,NaN,NaN,NaN
garantie,101,2,6 mois,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
annee,1497,NaN,NaN,NaN,14810.1,496237,0,1979,1998,2007,1.92019e+07


In [8]:
y_train.delai_vente.value_counts()

0    3027
2    2953
1    2900
Name: delai_vente, dtype: int64

Le jeu de données est très équilibré, chacune des 3 classes a une fréquence proche d’1/3.

# Création d'un modèle

Il est maintenant temps de créer un modele. Dans ce tutoriel nous allons construire une [Forêt Aléatoire](https://fr.wikipedia.org/wiki/For%C3%AAt_d'arbres_d%C3%A9cisionnels)

Pour ce faire nous utilisons les variables ```["poids", "prix", "nb_images", "longueur_image", "largeur_image", "categorie"]```.

Pour éviter le [surapprentissage](https://fr.wikipedia.org/wiki/Surapprentissage) et estimer les vraies performances de notre modèle nous allons utiliser le critère de [validation croisee](https://fr.wikipedia.org/wiki/Validation_crois%C3%A9e) méthode **k-fold** (cross-validation).

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Imputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.preprocessing import LabelEncoder

### Imputation des valeurs manquantes par la valeur "missing"

In [10]:
X_train.categorie.fillna('missing', inplace=True)
X_test.categorie.fillna('missing', inplace=True)

### Encodage des features catégorielles

Les algorithmes de machine learning s'attendent à avoir en entrée des **nombres**, et non pas des chaînes de caractères. C'est pourquoi nous transformons les **features catégorielles** en nombres, à l'aide de ```LabelEncoder()```

In [11]:
X_train.categorie.unique()

array(['mode', 'mobilier - deco', 'label selection', 'multimédia',
       'loisirs', 'enfance', 'librairie', 'culture - loisirs',
       'les coups de coeur des vendeurs', 'mobilier - deco - maison',
       'créations', 'missing'], dtype=object)

In [12]:
le = LabelEncoder()
X_train['categorie'] = le.fit_transform(X_train.categorie)
X_test['categorie'] = le.transform(X_test.categorie)

In [13]:
features = ["poids", "prix", "nb_images",
            "longueur_image", "largeur_image", "categorie"]

ppl = Pipeline([("imputer", Imputer(strategy='median')),
                ("clf", RandomForestClassifier(n_estimators=10))])

ppl.fit(X_train.loc[:, features], np.ravel(y_train))

pred_train = ppl.predict_proba(X_train.loc[:, features])
pred_cv = cross_val_predict(ppl, X_train.loc[:, features], np.ravel(y_train),
                            method='predict_proba', cv=5, n_jobs=-1)

# Calcul de l'erreur: logloss

In [14]:
from sklearn.metrics import log_loss 

In [15]:
print("LogLoss sur echantillon train:",log_loss(y_pred=pred_train, y_true=y_train))
print("LogLoss sur echantillon train (CV):",log_loss(y_pred=pred_cv, y_true=y_train))

LogLoss sur echantillon train: 0.298855984353
LogLoss sur echantillon train (CV): 3.41792486798


# Calcul des predictions

In [16]:
pred_test = ppl.predict_proba(X_test.loc[:, features])

In [17]:
df_submission = pd.DataFrame(pred_test, index=X_test.index)

# Soumission

## Possibilité n°1 : via l'API de QScore

1. Aller sur la plateforme [QScore](https://qscore.meilleurdatascientistdefrance.com) puis dans "Submissions" > "Submit from your Python Notebook"
2. Récuperer votre TOKEN
3. Le remplacer dans la fonction ci-dessous et l'exécuter

In [18]:
import io, math, requests

# Ne fonctionne qu'en Python3, voir commentaire ci-dessous pour Python2
def submit_prediction(df, sep=',', **kwargs):
    # TOKEN a recuperer sur la plateforme: "Submissions" > "Submit from your Python Notebook"
    TOKEN='<TON_TOKEN>'  
    URL='https://qscore.meilleurdatascientistdefrance.com/api/submissions'
    #buffer = io.BytesIO() # Python 2
    buffer = io.StringIO() # Python 3
    df.to_csv(buffer, sep=sep, **kwargs)
    buffer.seek(0)
    r = requests.post(URL, headers={'Authorization': 'Bearer {}'.format(TOKEN)},files={'datafile': buffer})
    if r.status_code == 429:
        raise Exception('Submissions are too close. Next submission is only allowed in {} seconds.'.format(int(math.ceil(int(r.headers['x-rate-limit-remaining']) / 1000.0))))
    if r.status_code != 200:
        raise Exception(r.text)

In [19]:
submit_prediction(df_submission, sep=',', index=True)

Exception: Cannot find any competition with token <TON_TOKEN>

## Possibilité n°2 : Soumettez un fichier CSV

1. Aller sur la plateforme [QScore](https://qscore.meilleurdatascientistdefrance.com) puis dans "Submissions" > "Submit with a file"
2. Déposer le fichier CSV

In [20]:
df_submission.to_csv("my_prediction.csv", index_label="id", header=['0', '1', '2'])